## **1) Importing Python Packages for GAN**


In [6]:
!nvidia-smi


Wed Nov 16 17:13:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# !pip install tensorflow

In [12]:
from keras.datasets import mnist #dataset of hand-written digits. 28x28 grayscale images

#Used to build NN
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Reshape, Flatten
from keras.layers import LeakyReLU
from keras.optimizers import Adam

import numpy as np
!mkdir generated_images #where we'll store generated images.

mkdir: cannot create directory ‘generated_images’: File exists


## **2) Variables for Neural Networks & Data**

In [13]:
img_width = 28
img_height = 28
channels = 1 #1 for grayscale, 3 for rgb
img_shape = (img_width, img_height, channels)
latent_dim = 100 #noise dimension (input for generator)
adam = Adam(learning_rate=0.0001) #variation on gradient descent.

## **3) Building Generator**





In [14]:
#Takes noise as input and outputs an image (width*height*channels numbers)
def build_generator():
  model = Sequential()
  #first layer
  model.add(Dense(256, input_dim=latent_dim)) #256 output neurons
  model.add(LeakyReLU(alpha=0.2)) #activation function (more stable than ReLU)
  model.add(BatchNormalization(momentum=0.8)) #improves speed, performance, and stability of NN

  #second layer
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  #third layer
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  #layer that reshapes back into (width, height, 1)
  model.add(Dense(np.prod(img_shape), activation='tanh')) #tanh activation function generally works well for output layer
  model.add(Reshape(img_shape))

  model.summary()
  return model

generator = build_generator()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_4 (Batc  (None, 256)              1024      
 hNormalization)                                      

Note: 28x28x1=784

## **4) Building Discriminator**

In [15]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape)) #flatten width*height*channel into 1 dimension

  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(256))
  model.add(Dense(1, activation='sigmoid')) #last layer will be a single number (probabililty of real/fake)

  model.summary()
  return model

discriminator = build_discriminator()

#set the loss function, gradient descent variation, and evluation metrics
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 256)               131328    
                                                                 
 dense_10 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


## **5) Connecting Neural Networks to build GAN**

In [16]:
GAN = Sequential()
#if we change the weights of the GAN, then only the generator weights will be updated, not discriminator.
#will make more sense later when training.
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)
GAN.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 28, 28, 1)         362000    
                                                                 
 sequential_2 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 895,505
Trainable params: 360,464
Non-trainable params: 535,041
_________________________________________________________________


## **6) Outputting Images**


In [17]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

## **7) Training GAN**

In [ ]:
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_, _) = mnist.load_data()
  print(X_train.shape)
  #Rescale data from [0,255] (grayscale) to [-1 and 1]
  X_train = X_train / 127.5 -1.
  print(X_train.shape)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1)) #vector batch_size ones
  fakes = np.zeros((batch_size, 1)) #vector batch_size zeros

  #training loop
  for epoch in range(epochs):
    #Get Random Batch of real images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    #Generate a batch of Fake Images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    #Train discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    #take average 
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    #inverse y label
    g_loss = GAN.train_on_batch(noise, valid)

    print("******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100* d_loss[1], g_loss))

    if(epoch % save_interval) == 0:
      save_imgs(epoch)

  # print(valid)


train(30000, batch_size=64, save_interval=200)

Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 4ms/step
******* 20007 [D loss: 0.374278, acc: 83.59%] [G loss: 2.050110]
2/2 [==============================] - 0s 5ms/step
******* 20008 [D loss: 0.454354, acc: 82.03%] [G loss: 1.817560]
2/2 [==============================] - 0s 4ms/step
******* 20009 [D loss: 0.393662, acc: 85.16%] [G loss: 1.801272]
2/2 [==============================] - 0s 5ms/step
******* 20010 [D loss: 0.417914, acc: 82.03%] [G loss: 1.781829]
2/2 [==============================] - 0s 7ms/step
******* 20011 [D loss: 0.304370, acc: 86.72%] [G loss: 2.407764]
2/2 [==============================] - 0s 5ms/step
******* 20012 [D loss: 0.507086, acc: 81.25%] [G loss: 2.259546]
2/2 [==============================] - 0s 6ms/step
******* 20013 [D loss: 0.364086, acc: 84.38%] [G loss: 2.176030]
2/2 [==============================] - 0s 6ms/step
******* 20014 [D loss: 0.485906, acc: 78.91%] [G loss: 2.086117]
2/2 [================

### **8) Making GIF**

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)